Simulation of matchgate circuits with matchgate noise:

In [8]:
from matchgates.generators import XY_circuit

n_qubits = 60  # number of qubits
dt = 0.1  # trotter step time interval
J = 0.5  # coupling strength
h = 0.23  # tranverse field
trotter_steps = 30  # number of trotter steps

# generate the XY circuit.
xy = XY_circuit(n_qubits=n_qubits, dt=dt, J=J, h=h, trotter_steps=trotter_steps)

# Avg. gate creation time per layer: 47.14273891644552

Step 1 / 30
Avg. gate creation time per layer: 1.503360915929079
Step 2 / 30
Avg. gate creation time per layer: 1.2486695619300008
Step 3 / 30
Avg. gate creation time per layer: 1.111996916277955
Step 4 / 30
Avg. gate creation time per layer: 2.1185102394083515
Step 5 / 30
Avg. gate creation time per layer: 2.064086149726063
Step 6 / 30
Avg. gate creation time per layer: 2.15334047190845
Step 7 / 30
Avg. gate creation time per layer: 2.141404946201614
Step 8 / 30
Avg. gate creation time per layer: 2.0863327236729674
Step 9 / 30
Avg. gate creation time per layer: 1.9704998331661854
Step 10 / 30
Avg. gate creation time per layer: 1.9780819456558674
Step 11 / 30
Avg. gate creation time per layer: 1.8840249051678588
Step 12 / 30
Avg. gate creation time per layer: 1.8128780658201624
Step 13 / 30
Avg. gate creation time per layer: 1.753658967773215
Step 14 / 30
Avg. gate creation time per layer: 1.7516206545489175
Step 15 / 30
Avg. gate creation time per layer: 1.7337502748395006
Step 16 / 3

Prepare initial state and observables:

In [9]:
from matchgates import Observable, ProductState

obsz = Observable(name="Z", qubits=[8], n_qubits=n_qubits)
obsyx = Observable(name="YX", qubits=[9, 10], n_qubits=n_qubits)

# prepare initial product state.
initial_state = ProductState.neel(even=True, n_qubits=n_qubits)

To add noise to the circuit, need to make a choice of noise for each gate in the circuit.
If all two qubit gates in the circuit have the same noise model, this noise model
can be set up in a simple way:

In [10]:
from matchgates import SingleGateNoise

p = 0.002
noise_channel = SingleGateNoise.matchgate_depolarizing(p=p)
# matchgate depolarizing is equal probability of perturbation by 
# any matchgate Pauli (Z, ZZ, XY, YX, YY, XX). 

noisy_xy = xy.add_two_qubit_uniform_noise(noise_channel)  # Add noise to circuit.

A noisy circuit can be simulated using the simulate_noisy method.
As the simulation is done by monte-carlo it is necessary to repeat
over some number of repetitions, and this can be distributed over 
multiple jobs. What will be returned is a single dictionary with the 
mean values for all observables in the list, averaged over the
repetitions specified.

In [12]:
from time import perf_counter

n_jobs = 20
n_jobs = 1
repetitions = 100

start = perf_counter()
results, hash_counts = noisy_xy.simulate_noisy(n_jobs, [obsz, obsyx], repetitions, initial_state)
end = perf_counter()

print("Results for XY circuit with noise:", results)

hash_counts_sorted_by_freq = sorted(list(hash_counts.values()), reverse=True)
trajectories_sorted_by_freq = sorted([(k, v,) for k, v in hash_counts.items()], key=lambda x: x[1], reverse=True)
all_I_string = "I" * len(list(hash_counts.keys())[0])
print("Sorted hash counts:")
print(hash_counts_sorted_by_freq)
print(f"Frequency of all-identity trajectory: {hash_counts[all_I_string]}")
print(f"Most common string: {trajectories_sorted_by_freq[0][0]}")
print(f"with frequency {hash_counts[trajectories_sorted_by_freq[0][0]]}")

print(f"Avg. time per run: {(end - start) / repetitions}s")

Results for XY circuit with noise: {'Z(8,)': np.float32(-0.036282398), 'YX(9, 10)': np.float32(0.21786572)}
Sorted hash counts:
[4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Frequency of all-identity trajectory: 0
Most common string: IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In practice, I find it most convenient to loop the above over a large number of iterations
and after each iterations append the results to a csv file. Then it is possible to access the
data over multiple iterations and compute statistics (mean, std, etc.) to check how well 
converged the results are.

In [ ]:
from time import time
import pandas as pd

start_time = time()

iterations = 4

for i in range(iterations):
    results = noisy_xy.simulate_noisy(n_jobs, [obsz, obsyx], repetitions, initial_state)
    new_results = {}
    for key in results.keys():
        new_results[key] = [results[key]]
    df = pd.DataFrame(new_results)
    if i == 0:
        df.to_csv(
            f"./XY_p{p}_N{n_qubits}_J{J}_h{h}_steps{trotter_steps}_rep{repetitions}.csv",
            mode="w",
            header=True,
            index=False,
        )
    else:
        df.to_csv(
            f"./XY_p{p}_N{n_qubits}_J{J}_h{h}_steps{trotter_steps}_rep{repetitions}.csv",
            mode="a",
            header=False,
            index=False,
        )
    print("Iteration: ", i)
    print("Time taken is ", time() - start_time)
    start_time = time()